In [1]:
import pandas as pd
pd.options.display.max_columns = None
from collections import Counter
import numpy as np

In [2]:
def unique_keyword_search(df_col: pd.Series, num_selected: int) -> list:
    '''
    Performs unique keyword search on a dataframe's column for its most common keywords

    :param pd.Series df_col: column of a pd.DataFrame (e.g. df['col'])
    :param int num_selected: number of keywords
    :return: list of keywords in decreasing occurrence
    :rtype: list
    '''
    keywords = []
    for entry in df_col:
        words = entry.split(',')
        [keywords.append(word) for word in words]
    sorted_keywords = Counter(keywords).most_common()
    output_keywords = []
    for i in range(num_selected):
        output_keywords.append(sorted_keywords[i][0])
    return output_keywords

In [3]:
org_df=pd.read_csv("org_final_joined_new.csv")
org_df.drop(org_df.iloc[:, 43:-54], inplace = True, axis = 1)
new_df= org_df[["Unnamed: 0", "uuid"]].copy()

/opt/anaconda3/envs/UBS/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (12,13,17,18,19,20,21,22,23,24,27,28,29,30,31,32,33,34,35,38,41,43,44,45,46,47,48,49,50,52,53,54,55,56,57,58,59,62,63,64,65,66,67,72,73,74,75,76,77,100,101,118) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
#D_country code
country_df= org_df[["country_code"]].copy()
country_df=pd.get_dummies(country_df, prefix=["country"], columns=["country_code"], drop_first=False)
new_df=new_df.join(country_df)

#E_Region
new_df["region"]= org_df["region"]

#F_CIty
city_df= org_df[["city"]].copy()
city_df=pd.get_dummies(city_df, prefix=["city"], columns=["city"], drop_first=False)
new_df=new_df.join(city_df)

#G_Status
status_df= org_df[["status"]].copy()
status_df=pd.get_dummies(status_df, prefix=["status"], columns=["status"], drop_first=False)

keep_col = ["status_operating", "status_closed"] # change the cols to keep
drop_row = [0, 1] # change the rows to drop
status_df = status_df[keep_col].drop(drop_row)
new_df=new_df.join(status_df)

#H:category_list & I: category_groups_list (p.s. there is no null cell)
category_df=org_df[["category_list","category_groups_list"]].copy()

#Concantante 2 columns
category_df["category_concat"]= category_df["category_list"]+ "," + category_df["category_groups_list"]

#keyword search
kw= unique_keyword_search(category_df["category_concat"], 100)
#kw[0]
name_arr= [[] for _ in range(100)] #When access name_arr[column][row]
counter=0 

for i in category_df["category_concat"]:    
    for j in range(len(kw)):
        if kw[j] in i: 
            #print(counter)
            name_arr[j].append(1)
        else: 
            name_arr[j].append(0)
    counter=counter+1

for i in range(100): 
    category_df["cat_"+kw[i]]=name_arr[i]
category_df= category_df.drop(columns=["category_list", "category_groups_list", "category_concat"])

new_df=new_df.join(category_df)

In [5]:
#O: Employee_count
def em_count_func(df): 
    if df["employee_count"]=="1-10" :
        return 1
    if df["employee_count"]=="11-50" :
        return 2
    if df["employee_count"]=="51-100" :
        return 3
    if df["employee_count"]=="101-250" :
        return 4
    if df["employee_count"]=="251-500" :
        return 5
    if df["employee_count"]=="501-1000" :
        return 6
    if df["employee_count"]=="1001-5000" :
        return 7
    if df["employee_count"]=="5001-10000" :
        return 8
    if df["employee_count"]=="10000+" :
        return 9
    if df["employee_count"]=="unknown" :
        return 0

Employee_count_df= org_df[["employee_count"]].copy()
Employee_count_df["employee_count_scaled"]= Employee_count_df.apply(em_count_func, axis=1)


keep_col = ["employee_count_scaled"] # change the cols to keep
drop_row = [0, 1] # change the rows to drop
Employee_count_df = Employee_count_df[keep_col].drop(drop_row)
new_df=new_df.join(Employee_count_df)


#P_primary_role
pri_role_df= org_df[["primary_role"]].copy()
pri_role_df=pd.get_dummies(pri_role_df, prefix=["pri_role"], columns=["primary_role"], drop_first=False)
new_df=new_df.join(pri_role_df)

#Q_num_exists 
num_exits_df= org_df[["num_exits"]].copy()
num_exits_df["num_exits"]=num_exits_df["num_exits"].fillna(0)

new_df=new_df.join(num_exits_df)

#R_lead_investor_name

dummy_df= org_df[["lead_investor_name"]].copy()
new_df=new_df.join(dummy_df)

#S_lead_investor_type

lead_invest_type_df=org_df[["lead_investor_type"]].copy()
lead_invest_type_df=pd.get_dummies(lead_invest_type_df, prefix=["lead_investor_type"], columns=["lead_investor_type"], drop_first=False)
#new_df=new_df.join(lead_invest_type_df)

#T_lead_investor_roles
lead_investor_role_df= org_df[["lead_investor_roles"]].copy()
investor=[]
company=[]
school=[]


inv_type_name_arr= ["investor", "company", "school"]
inv_type_arr= [investor, company, school]
lead_investor_role_df["lead_investor_roles"]= lead_investor_role_df["lead_investor_roles"].fillna(0)

for i in lead_investor_role_df["lead_investor_roles"]:
    if i==0: 
        inv_type_arr[0].append(0)
        inv_type_arr[1].append(0)
        inv_type_arr[2].append(0)
        continue
    for j in range(len(inv_type_name_arr)):
        if inv_type_name_arr[j] in i: 
            inv_type_arr[j].append(1)
        else: 
            inv_type_arr[j].append(0)

lead_investor_role_df["inv_investor"]= investor
lead_investor_role_df["inv_company"]= company
lead_investor_role_df["inv_school"]= school

keep_col = ["inv_investor", "inv_company", "inv_school"] # change the cols to keep
drop_row = [0, 1] # change the rows to drop
lead_investor_role_df = lead_investor_role_df[keep_col].drop(drop_row)
new_df=new_df.join(lead_investor_role_df)

In [6]:
#V_lead_investor_country_code

lead_country_df= org_df[["lead_investor_country_code"]].copy()
lead_country_df=pd.get_dummies(lead_country_df, prefix=["lead_investor_country"], columns=["lead_investor_country_code"], drop_first=False)
new_df=new_df.join(lead_country_df)

#W_Lead_investor_region
dummy_df= org_df[["lead_investor_region"]].copy()
new_df=new_df.join(dummy_df)

#X_lead_investor_city
lead_city_df= org_df[["lead_investor_city"]].copy()
lead_city_df=pd.get_dummies(lead_city_df, prefix=["lead_investor_city"], columns=["lead_investor_city"], drop_first=False)
new_df=new_df.join(lead_city_df)

#Y_lead_investor_type

lead_investor_type_df= org_df[["lead_investor_investor_types"]].copy()
lead_investor_type_df=pd.get_dummies(lead_investor_type_df, prefix=["investor_type"], columns=["lead_investor_investor_types"], drop_first=False)
lead_investor_type_df
new_df=new_df.join(lead_investor_type_df)

#Z_lead_investor_investment_count
lead_investor_count_df= org_df[["lead_investor_investment_count"]].copy()
lead_investor_count_df["lead_investor_investment_count"]=lead_investor_count_df["lead_investor_investment_count"].fillna(0)

new_df=new_df.join(lead_investor_count_df)

#AA_lead_investor_total_funding_usd
lead_investor_fund_df= org_df[["lead_investor_total_funding_usd"]].copy()
lead_investor_fund_df["lead_investor_total_funding_usd"]=lead_investor_fund_df["lead_investor_total_funding_usd"].fillna(0)

new_df=new_df.join(lead_investor_fund_df)

#AD_fund_rd_type
fund_rd_type_df= org_df[["fund_rd_type"]].copy()
fund_rd_type_df["fund_rd_type"]=fund_rd_type_df["fund_rd_type"].fillna(0)

new_df=new_df.join(fund_rd_type_df)

#AE_country_code

fund_country_df= org_df[["fund_rd_country_code"]].copy()
fund_country_df=pd.get_dummies(fund_country_df, prefix=["fund_country"], columns=["fund_rd_country_code"], drop_first=False)
new_df=new_df.join(fund_country_df)

#AF& AG region and statecode
dummy_df= org_df[["fund_rd_state_code", "fund_rd_region"]].copy()
new_df=new_df.join(dummy_df)

#AH_city

fund_city_df= org_df[["fund_rd_city"]].copy()
fund_city_df=pd.get_dummies(fund_city_df, prefix=["fund_country"], columns=["fund_rd_city"], drop_first=False)
new_df=new_df.join(fund_city_df)

#AI_fund_rd_raised_amount_usd (Discovered that if there is no entry for "type", there will be nan)

fund_raised_df= org_df[["fund_rd_raised_amount_usd"]].copy()
fund_raised_df=fund_raised_df.fillna(fund_raised_df.mean())

new_df=new_df.join(fund_raised_df)

# AN_ fund_rd_post_money_valuation_usd

fund_posted_df= org_df[["fund_rd_post_money_valuation_usd"]].copy()
fund_posted_df=fund_posted_df.fillna(fund_posted_df.mean())
new_df=new_df.join(fund_posted_df)

In [7]:
dfa = new_df

In [8]:
df = pd.read_csv('org_final_joined_new.csv')
df.drop(df.iloc[:, 0:43], inplace = True, axis = 1)
df.drop(df.iloc[:, -54:], inplace = True, axis = 1)
df.drop(columns=['partner_domain','investor_domain', 'ipo_stock_exchange_symbol'], inplace = True)

#Filling 0 for missing value for counts
df['partner_investment_count'] = df['partner_investment_count'].fillna(0)
df['investor_investment_count'] = df['investor_investment_count'].fillna(0)

#Fillna for total funds
df['investor_name'] = df['investor_name'].fillna("")
mean = df['investor_total_funding_usd'].mean()
#Selecting 0 or mean to be filled
def fillwhat(df): 
    if df['investor_name']=="" :
        return 0
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['investor_total_funding_usd','investor_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['investor_total_funding_usd'] = fill_df

#Fillna for acquisition prices
df['acquisitions_acquirer_name'] = df['acquisitions_acquirer_name'].fillna("")
mean = df['acquisitions_price_usd'].mean()
#Selecting 0 or mean to be filled
def fillwhat(df): 
    if df['acquisitions_acquirer_name']=="" :
        return 0
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['acquisitions_price_usd','acquisitions_acquirer_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['acquisitions_price_usd'] = fill_df

In [9]:
#Fillna for parent_organizations' ranking
mean = df['parent_org_rank'].mean()
#Selecting 0 or mean to be filled
def fillwhat(df): 
    if df['acquisitions_acquired_on']=="" :
        return 0
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['parent_org_rank','acquisitions_acquired_on']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['parent_org_rank'] = fill_df

one_hot_pn_tp = pd.get_dummies(df['partner_type'], prefix='partner_type')
one_hot_pn_rl = pd.get_dummies(df['partner_roles'], prefix='partner_roles')
one_hot_pn_cc = pd.get_dummies(df['partner_country_code'], prefix='partner_country')
one_hot_pn_ct = pd.get_dummies(df['partner_city'], prefix='partner_city')

# Drop the encoded columns
df = df.drop(['partner_type','partner_roles','partner_country_code','partner_city'],axis = 1)
# Join the encoded df
df = df.join(one_hot_pn_tp)
df = df.join(one_hot_pn_rl)
df = df.join(one_hot_pn_cc)
df = df.join(one_hot_pn_ct)

#types_series = df['partner_investor_types'].value_counts()
#print(types_series)

df['partner_investor_types'] = df['partner_investor_types'].fillna("")
investment_partner = []
angel = []

#If match append 1 otherwise append 0. Equivalent for one-hot encoding
for i in df['partner_investor_types']:
    if "investment_partner" in i:
        investment_partner.append(1)
    else:
        investment_partner.append(0)
    if "angel" in i:
        angel.append(1)
    else:
        angel.append(0)
        
#Drop the encoded column
df=df.drop(['partner_investor_types'],axis=1)
df['investment_partner'] = investment_partner
df['ange'] = angel

#types_series = df['investor_investor_types'].value_counts().nlargest(30)
#print(types_series)

one_hot_in_tp = pd.get_dummies(df['investor_type'], prefix='investor_type')
one_hot_in_rl = pd.get_dummies(df['investor_roles'], prefix='investor_roles')
one_hot_in_cc = pd.get_dummies(df['investor_country_code'], prefix='investor_country')
one_hot_in_ct = pd.get_dummies(df['investor_city'], prefix='investor_city')

# Drop the encoded columns
df = df.drop(['investor_type','investor_roles','investor_country_code','investor_city'],axis = 1)
# Join the encoded df
df = df.join(one_hot_in_tp)
df = df.join(one_hot_in_rl)
df = df.join(one_hot_in_cc)
df = df.join(one_hot_in_ct)

In [10]:
df['investor_investor_types'] = df['investor_investor_types'].fillna("")
venture_capital = []
micro_vc = []
private_equity_firm = []
corporate_venture_capital = []
entrepreneurship_program = []
government_office =[]
incubator = []
accelerator = []
angel_group = []
angel = []
family_investment_office = []
pension_funds = []
hedge_fund = []
fund_of_funds = []
investment_bank  = []
secondary_purchaser = []
investment_partner = []
venture_debt = []

inv_type_name_arr = ["venture_capital","micro_vc","private_equity_firm","corporate_venture_capital","entrepreneurship_program",\
                "government_office","incubator","accelerator","angel_group","angel","family_investment_office","pension_funds",\
                "hedge_fund", "fund_of_funds", "investment_bank", "secondary_purchaser", "investment_partner", "venture_debt"]
inv_type_arr = [venture_capital,micro_vc,private_equity_firm,corporate_venture_capital,entrepreneurship_program,\
                government_office,incubator,accelerator,angel_group,angel,family_investment_office,pension_funds,\
                hedge_fund, fund_of_funds, investment_bank, secondary_purchaser, investment_partner, venture_debt]

#Loop through both the df and the array. If match, the sub-array will append 1 otherwise 0 for one-hot encoding
for i in df['investor_investor_types']:
    for j in range (len(inv_type_name_arr)):
        if inv_type_name_arr[j] in i:
            inv_type_arr[j].append(1)
        else:
            inv_type_arr[j].append(0)
            
#Drop the encoded column
df=df.drop(['investor_investor_types'],axis=1)
for k in range (len(inv_type_name_arr)):
    df[inv_type_name_arr[k]] = inv_type_arr[k]
    
one_hot_ip_cc = pd.get_dummies(df['ipo_country_code'], prefix='ipo_country')
one_hot_ip_ct = pd.get_dummies(df['ipo_city'], prefix='ipo_city')
one_hot_ac_cc = pd.get_dummies(df['acquisitions_acquirer_country_code'], prefix='acquirer_country')
one_hot_ac_ct = pd.get_dummies(df['acquisitions_acquirer_city'], prefix='acquirer_city')

# Drop the encoded columns
df = df.drop(['ipo_country_code','ipo_city','acquisitions_acquirer_country_code','acquisitions_acquirer_city'],axis = 1)
# Join the encoded df
df = df.join(one_hot_ip_cc)
df = df.join(one_hot_ip_ct)
df = df.join(one_hot_ac_cc)
df = df.join(one_hot_ac_ct)

#Fill mean values for datetime. "Investor_founded_on" and "acquisitions_acquired_on"

#Firstly, convert the dates into pandas-compatible datetime value
df['investor_founded_on'] = pd.to_datetime(df['investor_founded_on'], infer_datetime_format=True)

#Next, change datatime to their corresponding integer value to compute the mean
df['investor_founded_on'] = df['investor_founded_on'].apply(lambda x: x.value)
mean = df['investor_founded_on'].mean()

def fillwhat(df): 
    if df['investor_name'] == "" :
        return ""
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['investor_founded_on','investor_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['investor_founded_on'] = fill_df

#Finally, change the integer back to datetime (optional)
df['investor_founded_on'] = df['investor_founded_on'].apply(pd.Timestamp)

#Again, for acquisitions date

#Firstly, convert the dates into pandas-compatible datetime value
df['acquisitions_acquired_on'] = pd.to_datetime(df['acquisitions_acquired_on'], infer_datetime_format=True)

#Next, change datatime to their corresponding integer value to compute the mean
df['acquisitions_acquired_on'] = df['acquisitions_acquired_on'].apply(lambda x: x.value)
mean = df['acquisitions_acquired_on'].mean()

def fillwhat(df): 
    if df['acquisitions_acquirer_name'] == "" :
        return ""
    else :
        return mean

fill_df = df.copy()
fill_df = fill_df[['acquisitions_acquired_on','acquisitions_acquirer_name']].apply(fillwhat, axis=1)
fill_df = pd.DataFrame(fill_df)
df['acquisitions_acquired_on'] = fill_df

#Finally, change the integer back to datetime (optional)
df['acquisitions_acquired_on'] = df['acquisitions_acquired_on'].apply(pd.Timestamp)

In [11]:
dfb = df

In [12]:
df=pd.read_csv("org_final_joined_new.csv")
df.drop(df.iloc[:, 0:80], inplace = True, axis = 1)
# some cols will be deleted, for being too complicated / containing little useful info
col_to_drop = ['event_short_description', 'event_venue_name', 'event_description', 'person_personal_featured_job_organization_name']
df.drop(columns=col_to_drop, inplace=True)
# some cols will use 0 as the fillna() value
col_nan_to_zero = ['person', 'organization', 'speaker', 'sponsor', 'exhibitor', 'organizer', 'contestant', 'job_executive', 'job_employee', 'job_board_member', 'job_advisor', 'job_board_observer', 'person_person', 'person_organization', 'person_speaker', 'person_sponsor', 'person_exhibitor', 'person_organizer', 'person_contestant']
for col in col_nan_to_zero:
    df[col].fillna(0)

In [13]:
# CD: event_rank
event_rank_mean = df['event_rank'].mean()
df['event_rank'] = df['event_rank'].fillna(event_rank_mean)

# CF, CG: event_started_on, event_ended_on
event_date = ['event_started_on', 'event_ended_on']
has_event = ~df['event_names'].isnull()

for col in event_date:
    df[col] = pd.to_datetime(df[col])
    mean = df[col].mean()
    df.loc[has_event,col] = df.loc[has_event,col].fillna(mean)
    
# Stardust ver one-hot for country_code, stock_exchange_symbol
def onehot_encoder(df: pd.DataFrame, col_name: str, num_selected: int) -> pd.DataFrame:
    '''
    Performs one-hot encoding on a dataframe's column for its values with most occurrences

    :param pd.DataFrame df: dataframe to be processed
    :param str col_name: name of the encoded column
    :param int num_selected: number of values with most occurrences
    :return: processed dataframe
    :rtype: pd.DataFrame
    '''
    series = df[col_name].value_counts()
    selected_col_name = series.head(num_selected).index.tolist()

    for item in selected_col_name:
        df[item] = np.where(df[col_name] == item, 1, 0) # whenever df[col_name] == cat replace it with 1 else 0
        # df[item].add_prefix(col_name + "_")
    df.drop(columns=col_name)
    return df

# CJ, CL: event_country_code, event_city
df = onehot_encoder(df, 'event_country_code', 20)
df = onehot_encoder(df, 'event_city', 20)

# DE, DF, DH: person_personal_gender, person_personal_country_code, person_personal_city
df = onehot_encoder(df, 'person_personal_gender', 2)
df = onehot_encoder(df, 'person_personal_country_code', 20)
df = onehot_encoder(df, 'person_personal_city', 20)

In [14]:
# DK: person_degree_degree_type
# use keyword to search for relevant degrees
bachelor_keyword = ['Bachelor', 'Degree', 'BS', 'BSc', 'B.S.', 'Bsc', 'B.S',
'BENG', 'BEng', 'B.Eng.', 'Beng', 'B.Eng', 'BE', 'B.E', 'BA', 'Ba', 'B.A.', 'B.A', 'A.B.', 'AB',
'BBA', 'B.B.A.', 'B.B.A', 'B.Tech', 'B.Tech.', 'B.Com.', 'J.D.', 'JD', 'Juris Doctor']

master_keyword = ['Master', 'Postgraduate', 'Graduate', 'MPHIL', 'MPhil', 'Mphil', 'M.Phil.', 'M.phil', 'M.Phil', 'M.S',
'MS', 'MSc', 'M.Sc', 'Msc', 'MENG', 'MEng', 'M.Eng.', 'M.eng.',
'MA', 'M.A', 'MBA', 'M.B.A.', 'M.B.A', 'Mba', 'M.BA.', 'M.Ba.', 'LLM']

phd_keyword = ['PHD', 'Phd', 'PhD', 'P.HD', 'P.Hd', 'P.hd', 'P.H.D', 'Ph.D.', 'Ph.D.', 'PhD']

df['person_degree_degree_type'].fillna(value='N/A', inplace=True)
df.loc[df['person_degree_degree_type'].str.contains('|'.join(phd_keyword)),'person_degree_degree_type']='PhD'
df.loc[df['person_degree_degree_type'].str.contains('|'.join(master_keyword)),'person_degree_degree_type']='Master'
df.loc[df['person_degree_degree_type'].str.contains('|'.join(bachelor_keyword)),'person_degree_degree_type']='Bachelor'

degree_type = []

# perform ordinal encoding: bachelor = 1, master = 2, PhD = 3, no/others = 0
for degree in df['person_degree_degree_type']:
    if degree == 'Bachelor':
        degree_type.append(1)
    elif degree == 'Master':
        degree_type.append(2)
    elif degree == 'PhD':
        degree_type.append(3)
    else:
        degree_type.append(0)
        
df['person_degree_degree_type'] = degree_type


# Stardust ver unique keyword search
def unique_keyword_search(df_col: pd.Series, num_selected: int) -> list:
    '''
    Performs unique keyword search on a dataframe's column for its most common keywords

    :param pd.Series df_col: column of a pd.DataFrame (e.g. df['col'])
    :param int num_selected: number of keywords
    :return: list of keywords in decreasing occurrence
    :rtype: list
    '''
    keywords = []
    for entry in df_col:
        words = entry.split(',')
        words = entry.split('&')
        words = entry.split('and')
        [keywords.append(word) for word in words]
    sorted_keywords = Counter(keywords).most_common()
    output_keywords = []
    for i in range(num_selected):
        output_keywords.append(sorted_keywords[i][0])
    return output_keywords

In [15]:
# # can't run

# pls refer to https://stackoverflow.com/questions/42224700/attributeerror-float-object-has-no-attribute-split
# Reason: NaN data presents
# To solve: Fillna("") .. fill all NaN values with empty string first

#df['person_degree_subject'] = df['person_degree_subject'].fillna("")

#subject_keywords = unique_keyword_search(df['person_degree_subject'], 50)


# onehot_encoder_v2(df, 'person_degree_subject', subject_keywords)
# df.head(50)

# DL_person_degree_subject
# df['person_degree_subject'].value_counts()
# df['person_degree_subject'].value_counts().head(50)

In [ ]:
# onehot_encoder_v2(df, 'person_degree_subject', subject_keywords)
# df.head(50)

In [ ]:
# # DM, DN: person_degree_started_on, person_degree_completed_on
# degree_date = ['person_degree_started_on', 'person_degree_completed_on']

# has_degree = np.where(df['person_degree_degree_type'] != 0, 1, 0)

# for col in degree_date:
#     df[col] = pd.to_datetime(df[col])
#     mean = df[col].mean()
#     df.loc[has_degree,col] = df.loc[has_degree,col].fillna(value=mean, inplace=True)

In [ ]:
# DO: person_degree_is_completed
df['person_degree_is_completed'].fillna(value=0, inplace=True)
df['person_degree_is_completed'] = df['person_degree_is_completed']*1

In [ ]:
# DQ: person_event_rank
person_event_rank_mean = df['person_event_rank'].mean()
df['person_event_rank'] = df['person_event_rank'].fillna(value=person_event_rank_mean, inplace=True)

In [ ]:
# DQ, DR: person_event_started_on, person_event_ended_on
event_date = ['person_event_started_on', 'person_event_ended_on']
has_event = ~df['person_event_name'].isnull()

for col in event_date:
    df[col] = pd.to_datetime(df[col])
    mean = df[col].mean()
    df.loc[has_event,col] = df.loc[has_event,col].fillna(value=mean, inplace=True)

In [ ]:
# DV: person_event_city
df = onehot_encoder(df, 'person_event_city', 20)

In [16]:
dfc = df

In [17]:
df = dfa.join(dfb)
df = df.join(dfc)

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018183 entries, 0 to 1018182
Columns: 7379 entries, Unnamed: 0 to Vancouver
dtypes: datetime64[ns](4), float64(41), int64(182), object(42), uint8(7110)
memory usage: 8.8+ GB


In [20]:
num_col = len(df.columns)
num_col_100 = int(num_col/100)
for i in range (1,num_col_100):
    print(df.iloc[:,100*(i-1):100*i].info())
print(df.iloc[:,num_col_100:].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1018183 entries, 0 to 1018182
Data columns (total 100 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   Unnamed: 0   1018183 non-null  int64 
 1   uuid         1018183 non-null  object
 2   country_AFG  1018183 non-null  uint8 
 3   country_AGO  1018183 non-null  uint8 
 4   country_AIA  1018183 non-null  uint8 
 5   country_ALB  1018183 non-null  uint8 
 6   country_AND  1018183 non-null  uint8 
 7   country_ARE  1018183 non-null  uint8 
 8   country_ARG  1018183 non-null  uint8 
 9   country_ARM  1018183 non-null  uint8 
 10  country_AUS  1018183 non-null  uint8 
 11  country_AUT  1018183 non-null  uint8 
 12  country_AZE  1018183 non-null  uint8 
 13  country_BAH  1018183 non-null  uint8 
 14  country_BEL  1018183 non-null  uint8 
 15  country_BEN  1018183 non-null  uint8 
 16  country_BGD  1018183 non-null  uint8 
 17  country_BGR  1018183 non-null  uint8 
 18  country_BHR  1018183 

In [ ]:
compression_opts = dict(method='zip', archive_name='data_cleaned.csv') 
df.to_csv('data_cleaned.csv', index= False, encoding='utf-8', compression=compression_opts)